In [1]:
import torch
import pandas as pd

In [2]:
# 1. embedding vector 텐서 파일 불러와서 shape 확인 : torch.Size([12491, 3840])
concatenated_embedding_vectors = torch.load('../data/JobDescription/tensor_data.pth')
concatenated_embedding_vectors.shape

torch.Size([12491, 3840])

In [3]:
# 2. url 가져와서 텐서와 붙이는 함수 정의
def vector_with_url(tensor):
    url = pd.read_csv('../data/JobDescription/pre_result.csv')['출처 URL']
    vector_with_url = {}
    for row in range(len(url)):
        vector_with_url[url[row]] = tensor[row]
    return vector_with_url

In [4]:
# 3. URL 쌍 간의 코사인 유사도를 계산하여 DataFrame을 생성하는 함수 정의
def cos_similarity_df(data):
    url1_list = []
    url2_list = []
    cos_list = []

    # 모든 URL 쌍에 대해 코사인 유사도 계산
    for url1, tensor1 in data.items():
        for url2, tensor2 in data.items():
            if url1 != url2: # 자기 자신과의 계산은 뺍니다!
                url1_list.append(url1)
                url2_list.append(url2)
                cosine_similarity = torch.cosine_similarity(tensor1, tensor2, dim=0)
                cos_list.append(cosine_similarity.item())

    # DataFrame 생성
    df = pd.DataFrame({
        'url1': url1_list,
        'url2': url2_list,
        'cosine-similarity': cos_list
    })

    return df


## 3개 데이터로만 TEST해본 결과 잘 돌아갑니다.

In [5]:
# 임베딩 벡터에 url을 붙여줍니다.
new_embedding_vectors = vector_with_url(concatenated_embedding_vectors)
new_embedding_vectors

{'https://www.wanted.co.kr/wd/167436': tensor([-0.0482, -0.0684,  0.1829,  ..., -0.2828,  0.1753,  0.2709]),
 'https://www.wanted.co.kr/wd/167433': tensor([ 0.0370, -0.0011,  0.1116,  ..., -0.3179,  0.2235,  0.1840]),
 'https://www.wanted.co.kr/wd/167134': tensor([ 0.0258,  0.0194,  0.1455,  ..., -0.2688,  0.2132,  0.1755]),
 'https://www.wanted.co.kr/wd/167133': tensor([ 0.0579, -0.0178,  0.1975,  ..., -0.2404,  0.1942,  0.1564]),
 'https://www.wanted.co.kr/wd/166922': tensor([ 0.1314, -0.0987,  0.1413,  ..., -0.4228,  0.3011,  0.2964]),
 'https://www.wanted.co.kr/wd/166844': tensor([-0.0620,  0.0326,  0.1237,  ..., -0.2571,  0.1726,  0.2064]),
 'https://www.wanted.co.kr/wd/166616': tensor([ 8.5670e-02,  2.7298e-05,  9.1450e-02,  ..., -3.7678e-01,
          2.5283e-01,  2.1942e-01]),
 'https://www.wanted.co.kr/wd/166030': tensor([ 0.1235, -0.0724,  0.1593,  ..., -0.0708, -0.0023,  0.0499]),
 'https://www.wanted.co.kr/wd/165816': tensor([ 0.0185,  0.0038,  0.1527,  ..., -0.1983,  0.238

In [6]:
# 이 중 3개 데이터만 선택해서 테스트해보겠습니다.
selected_keys = list(new_embedding_vectors.keys())[:3]
selected_data = {key: new_embedding_vectors[key] for key in selected_keys}
selected_data

{'https://www.wanted.co.kr/wd/167436': tensor([-0.0482, -0.0684,  0.1829,  ..., -0.2828,  0.1753,  0.2709]),
 'https://www.wanted.co.kr/wd/167433': tensor([ 0.0370, -0.0011,  0.1116,  ..., -0.3179,  0.2235,  0.1840]),
 'https://www.wanted.co.kr/wd/167134': tensor([ 0.0258,  0.0194,  0.1455,  ..., -0.2688,  0.2132,  0.1755])}

In [7]:
# DataFrame 생성 및 csv 파일 생성
col_name = '전체컬럼'

df = cos_similarity_df(selected_data)
df.to_csv(f"cosine_cal_{col_name}_test.csv",index=False)

### Tensor를 5등분하여, 하나의 col 혹은 두 개 이상의 col을 선택할 수 있도록 하는 부분입니다.

In [8]:
# 텐서를 5개의 부분으로 나눕니다.
tensor_parts = torch.chunk(concatenated_embedding_vectors, chunks=5, dim=1)

# 각 부분의 크기 출력 : (torch.Size([12491, 768]))인지 확인
for i, part in enumerate(tensor_parts):
    print(f"Part {i} size: {part.size()}")
print("====================")

# 나누어진 텐서 확인
for i, part in enumerate(tensor_parts):
    print(f"Part {i}:")
    print(part)

Part 0 size: torch.Size([12491, 768])
Part 1 size: torch.Size([12491, 768])
Part 2 size: torch.Size([12491, 768])
Part 3 size: torch.Size([12491, 768])
Part 4 size: torch.Size([12491, 768])
Part 0:
tensor([[-0.0482, -0.0684,  0.1829,  ..., -0.2137,  0.1857,  0.1891],
        [ 0.0370, -0.0011,  0.1116,  ..., -0.1033,  0.0084,  0.0612],
        [ 0.0258,  0.0194,  0.1455,  ..., -0.1985,  0.0530,  0.0386],
        ...,
        [ 0.0802, -0.0974,  0.1981,  ..., -0.2107,  0.1324,  0.2148],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478]])
Part 1:
tensor([[ 0.0473,  0.0635,  0.0188,  ..., -0.1055,  0.0149, -0.0336],
        [-0.1230, -0.1551,  0.1149,  ..., -0.3117,  0.1706,  0.0673],
        [ 0.0788, -0.0896,  0.0954,  ..., -0.3110,  0.1475,  0.1109],
        ...,
        [ 0.2323, -0.0134,  0.1699,  ..., -0.3207,  0.1293,  0.1575],
        [ 0.1135,  0.0599,  0.1207,  ..., -0.1289,  0.0667,  0.1798

## 실제 코드 작업하실 때는 여기 아래부터 수정해주시면 됩니다!

In [9]:
# ['자격요건', '우대조건', '복지', '회사소개', '주요업무'] 하나씩 분리합니다. 차례대로 part 0,1,2,3,4입니다.
# ex) ['자격요건'] 하나를 선택하는 경우 tensor_parts[0]을 입력해주시면 됩니다.
one_tensor = tensor_parts[0]

# one_tensor 사이즈 확인
print(f"Combined Tensor Size: {one_tensor.size()}")
print("Combined Tensor:")
print(one_tensor)

Combined Tensor Size: torch.Size([12491, 768])
Combined Tensor:
tensor([[-0.0482, -0.0684,  0.1829,  ..., -0.2137,  0.1857,  0.1891],
        [ 0.0370, -0.0011,  0.1116,  ..., -0.1033,  0.0084,  0.0612],
        [ 0.0258,  0.0194,  0.1455,  ..., -0.1985,  0.0530,  0.0386],
        ...,
        [ 0.0802, -0.0974,  0.1981,  ..., -0.2107,  0.1324,  0.2148],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3600,  0.2164,  0.2478]])


In [10]:
# 2개 이상의 col을 선택할 때 텐서를 합쳐주는 부분입니다. 
# ex) ['자격요건','주요업무']를 합치는 경우 part 0,4를 합쳐주시면 됩니다.
combined_tensor = torch.cat([tensor_parts[0], tensor_parts[4]], dim=1)

# combined_tensor 사이즈 확인
print(f"Combined Tensor Size: {combined_tensor.size()}")
print("Combined Tensor:")
print(combined_tensor)


Combined Tensor Size: torch.Size([12491, 1536])
Combined Tensor:
tensor([[-0.0482, -0.0684,  0.1829,  ..., -0.2828,  0.1753,  0.2709],
        [ 0.0370, -0.0011,  0.1116,  ..., -0.3179,  0.2235,  0.1840],
        [ 0.0258,  0.0194,  0.1455,  ..., -0.2688,  0.2132,  0.1755],
        ...,
        [ 0.0802, -0.0974,  0.1981,  ..., -0.3072,  0.1223,  0.2173],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3288,  0.1136,  0.1573],
        [ 0.1956, -0.1600,  0.2822,  ..., -0.3288,  0.1136,  0.1573]])


In [11]:
# 임베딩 벡터에 url을 붙여줍니다.
new_embedding_vectors = vector_with_url(여기에 만든 one_tensor OR combined_tensor를 넣어주세요)
new_embedding_vectors

# DataFrame 생성 및 csv 파일 생성
# col_name = '주요업무' <- 실제 작업 시에는 이런 식으로 넣어주세요! 
col_name = '전체column'

df = cos_similarity_df(selected_data)
df.to_csv(f"cosine_cal_{col_name}_test.csv",index=False)